## Importing Libraries

In [1]:
import pandas as pd
from pypfopt.expected_returns import mean_historical_return
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import discrete_allocation
from pypfopt import expected_returns
from pypfopt import risk_models
from pypfopt import EfficientFrontier
from pypfopt import objective_functions
from pypfopt import base_optimizer
from pypfopt.discrete_allocation import DiscreteAllocation
from pypfopt.hierarchical_portfolio import HRPOpt

## Getting the Data

In [2]:
# Define the ticker list
tickers_list = ['AAPL', 'WMT', 'MU', 'BA','GOOG','BABA','GE','AMD','BAC','GM','T','UAA','XOM','RRC','PFE','JPM','SBUX']

# Fetch the data
import yfinance as yf
data = yf.download(tickers_list,'2022-6-1')['Adj Close']
#print(data.iloc[-1])
# Print first 5 rows of the data
#print(data.head())
#print(data.tail())

[*********************100%***********************]  17 of 17 completed


## Markowitz with Sharpe maximization

In [3]:
# Prepare historical price data for assets
#historical_prices = pd.read_csv('historical_prices.csv', index_col=0, parse_dates=True)
historical_prices = data
#target_volatility = 0.2


# Calculate expected returns
returns = expected_returns.mean_historical_return(historical_prices)
cov_matrix = risk_models.sample_cov(historical_prices)

# Create an instance of EfficientFrontier
ef = EfficientFrontier(returns, cov_matrix,weight_bounds = (0,0.2))

# Optimize for maximum Sharpe ratio
weights = ef.max_sharpe()

#  print weights
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

OrderedDict([('AAPL', 0.13952), ('AMD', 0.0), ('BA', 0.2), ('BABA', 0.0), ('BAC', 0.0), ('GE', 0.2), ('GM', 0.0), ('GOOG', 0.0), ('JPM', 0.0), ('MU', 0.0), ('PFE', 0.0), ('RRC', 0.0), ('SBUX', 0.2), ('T', 0.0), ('UAA', 0.0), ('WMT', 0.2), ('XOM', 0.06048)])


In [4]:
#to know the expected performance of the portfolio with optimal weights w
ef.portfolio_performance(verbose=True)

Expected annual return: 43.1%
Annual volatility: 21.7%
Sharpe Ratio: 1.89


(0.43111261720920735, 0.21746798265070416, 1.8904512388360823)

In [5]:
# Calculate the discrete allocation of assets based on the optimal weights
latest_prices = historical_prices.iloc[-1]  # Latest prices for the assets
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.lp_portfolio()

/home/bhavya/anaconda3/lib/python3.10/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [6]:
print("Optimal Allocation:", allocation)

Optimal Allocation: {'AMD': 1, 'BA': 9, 'BABA': 1, 'BAC': 3, 'GE': 20, 'GM': 2, 'GOOG': 1, 'JPM': 1, 'MU': 1, 'PFE': 2, 'RRC': 3, 'SBUX': 21, 'T': 6, 'UAA': 12, 'WMT': 13, 'XOM': 6}


## Markowitz with Target Volatility

In [7]:
# Prepare historical price data for assets
#historical_prices = pd.read_csv('historical_prices.csv', index_col=0, parse_dates=True)
historical_prices = data
target_volatility = 0.2


# Calculate expected returns
returns = expected_returns.mean_historical_return(historical_prices)
cov_matrix = risk_models.sample_cov(historical_prices)

# Create an instance of EfficientFrontier
ef_tv = EfficientFrontier(returns, cov_matrix,weight_bounds = (0,0.2))

ef_tv.efficient_risk(target_volatility)

# Optimize for maximum Sharpe ratio
#weights = ef.max_sharpe()

#  print weights
cleaned_weights_tv = ef_tv.clean_weights()
print(cleaned_weights)

OrderedDict([('AAPL', 0.13952), ('AMD', 0.0), ('BA', 0.2), ('BABA', 0.0), ('BAC', 0.0), ('GE', 0.2), ('GM', 0.0), ('GOOG', 0.0), ('JPM', 0.0), ('MU', 0.0), ('PFE', 0.0), ('RRC', 0.0), ('SBUX', 0.2), ('T', 0.0), ('UAA', 0.0), ('WMT', 0.2), ('XOM', 0.06048)])


In [8]:
#to know the expected performance of the portfolio with optimal weights w
ef_tv.portfolio_performance(verbose=True)

Expected annual return: 36.3%
Annual volatility: 20.0%
Sharpe Ratio: 1.71


(0.3626379233916832, 0.2000000002091851, 1.7131896151665473)

In [9]:
# Calculate the discrete allocation of assets based on the optimal weights
latest_prices = historical_prices.iloc[-1]  # Latest prices for the assets
da = DiscreteAllocation(cleaned_weights_tv, latest_prices, total_portfolio_value=10000)
allocation_tv, leftover = da.lp_portfolio()

/home/bhavya/anaconda3/lib/python3.10/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [10]:
print("Optimal Allocation:", allocation_tv)

Optimal Allocation: {'BA': 7, 'BAC': 1, 'GE': 19, 'GM': 1, 'PFE': 21, 'RRC': 1, 'SBUX': 20, 'T': 15, 'UAA': 4, 'WMT': 13, 'XOM': 10}


## Markowitz with Monthly Rebalancing keeping Original Weights Only

Currently getting ValueError: at least one of the assets must have an expected return exceeding the risk-free rate


In [11]:
# Create an empty DataFrame to store allocation results
allocation_history = pd.DataFrame(columns=tickers_list)

# Set initial portfolio value
portfolio_value = 10000

# Set rebalancing frequency (in months)
rebalancing_frequency = 1

# Iterate over each month
for i in range(0, len(historical_prices), rebalancing_frequency):
    # Get historical prices for the current month
    prices = historical_prices.iloc[i:i+rebalancing_frequency]

    # Calculate expected returns and covariance matrix
    returns = expected_returns.mean_historical_return(prices)
    cov_matrix = risk_models.sample_cov(prices)

    # Create an instance of EfficientFrontier
    ef = EfficientFrontier(returns, cov_matrix, weight_bounds=(0, 0.2))

    # Optimize for maximum Sharpe ratio
    weights = ef.max_sharpe()

    # Clean the weights for assets below the lower weight bound
    cleaned_weights = ef.clean_weights()

    # Calculate the discrete allocation of assets based on the optimal weights
    latest_prices = prices.iloc[-1]  # Latest prices for the assets
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
    allocation, _ = da.lp_portfolio()

    # Store the allocation for the current month
    allocation_history = allocation_history.append(allocation, ignore_index=True)

# Print the allocation history
print(allocation_history)

/home/bhavya/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/bhavya/.local/lib/python3.10/site-packages/numpy/core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/home/bhavya/.local/lib/python3.10/site-packages/pandas/core/frame.py:10222: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/home/bhavya/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/home/bhavya/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2705: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


ValueError: at least one of the assets must have an expected return exceeding the risk-free rate

### Test // Combining all above approaches into a single function

In [ ]:
def optimize_portfolio(historical_prices, objective, target=None, rebalance=False):
    # Calculate expected returns
    returns = expected_returns.mean_historical_return(historical_prices)
    cov_matrix = risk_models.sample_cov(historical_prices)

    # Create an instance of EfficientFrontier
    ef = EfficientFrontier(returns, cov_matrix, weight_bounds=(0, 0.2))

    if objective == "max_sharpe":
        # Optimize for maximum Sharpe ratio
        weights = ef.max_sharpe()
    elif objective == "target_volatility":
        # Optimize for target volatility
        ef.efficient_risk(target)
        weights = ef.clean_weights()
    else:
        raise ValueError("Invalid optimization objective. Please choose 'max_sharpe' or 'target_volatility'.")

    # Print weights
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)

    # Portfolio performance
    ef.portfolio_performance(verbose=True)

    # Calculate the discrete allocation of assets based on the optimal weights
    latest_prices = historical_prices.iloc[-1]  # Latest prices for the assets
    da = discrete_allocation.DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=10000)
    allocation, leftover = da.lp_portfolio()

    print("Optimal Allocation:", allocation)

    if rebalance:
        # Rebalance monthly
        rebalance_date = historical_prices.index[-1] + timedelta(days=30)
        historical_prices_rebalanced = historical_prices.loc[:rebalance_date]
        optimize_portfolio(historical_prices_rebalanced, objective, target, rebalance=True)


# Getting the Data
tickers_list = ['AAPL', 'WMT', 'MU', 'BA', 'GOOG', 'BABA', 'GE', 'AMD', 'BAC', 'GM', 'T', 'UAA', 'XOM', 'RRC', 'PFE', 'JPM',
                'SBUX']
data = yf.download(tickers_list, '2022-6-1')['Adj Close']

# Markowitz with Sharpe maximization
historical_prices = data
optimize_portfolio(historical_prices, objective="max_sharpe", rebalance=False)

# Markowitz with Target Volatility
historical_prices = data
target_volatility = 0.2
optimize_portfolio(historical_prices, objective="target_volatility", target=target_volatility, rebalance=True)

## Markowitz with Monthly Rebalancing - With Original Weights Revising every 12 months